In [1]:
import re
import json
import pandas as pd
from glob import glob
from os.path import join,basename
from msp_tables import prepare_tables, log
# make things prettier
from rich import print
from rich.progress import track, Progress

log.setLevel("ERROR")

bidsdir = '/mnt/bulk-vega/paulkuntke/mspaths'



First we filter out all available patient-IDs as well as the dates of examination

In [2]:
mpis = [ basename(sub).replace('sub-','')  for sub in glob(join(bidsdir, 'sub-*'))]

Read all the session-dates - as defined in the BIDS-Structure

In [3]:
sessionfiles = glob(join(bidsdir, 'sub-*', 'sub-*_sessions.tsv'))

sessions_df = pd.DataFrame()

subject_pattern = re.compile(r"sub-(\d+)_")


for sesfile in track(sessionfiles):
    df = pd.read_csv(sesfile, delimiter='\t')
    df["subject"] = re.findall(subject_pattern, basename(sesfile))[0]
    sessions_df = pd.concat((sessions_df, df))

del(df)
del(sessionfiles)

sessions_df.acq_time = pd.to_datetime(sessions_df.acq_time)



Output()

In [4]:
sessions_df.drop_duplicates()

,session_id,acq_time,subject
0,ses-001,2019-05-03,100052186
1,ses-002,2020-02-22,100052186
0,ses-001,2017-02-09,100006427
1,ses-002,2017-06-08,100006427
0,ses-001,2020-11-08,500002712
...,...,...,...
1,ses-002,2017-11-24,100003594
2,ses-003,2019-12-29,100003594
0,ses-001,2019-06-17,500003493
1,ses-002,2020-07-10,500003493


Now- lets read all tables ( _all_ is defined in `column_names.json`)

In [6]:
with open("column_names.json", "r") as file:
    table_data = json.load(file)
prepared_tables = prepare_tables('888MS001 & 888MS002/Data Tables/', bidsdir, table_data)
prepared_tables_hc = prepare_tables('888MS005/Data Tables/', bidsdir, table_data)

/home/paulkuntke/mspaths/msp_tables.py:74: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  f_df = pd.read_csv(f, encoding="cp1252") # Some tables contain Chars that are not readable with default utf-8 codepage
/home/paulkuntke/mspaths/msp_tables.py:74: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  f_df = pd.read_csv(f, encoding="cp1252") # Some tables contain Chars that are not readable with default utf-8 codepage
/home/paulkuntke/mspaths/msp_tables.py:74: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  f_df = pd.read_csv(f, encoding="cp1252") # Some tables contain Chars that are not readable with default utf-8 codepage
/home/paulkuntke/mspaths/msp_tables.py:74: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  f_df = pd.read_csv(f, encoding="cp1252") # Some tables contain 

In [7]:
prepared_tables["MSPT Sociodemographics"]

,mpi,site,sex,file,race,encounter_date,ethnicity,age_at_first_symptom,age_at_diagnosis
1,100000012,275.0,female,mspt_sociodemographics_v003.csv,white,1.461329e+09,not_hispanic_or_latino,40.0,40.0
4,100000017,275.0,female,mspt_sociodemographics_v003.csv,white,1.461402e+09,not_hispanic_or_latino,33.0,33.0
5,100000021,275.0,male,mspt_sociodemographics_v003.csv,white,1.461487e+09,not_hispanic_or_latino,30.0,30.0
6,100000023,275.0,male,mspt_sociodemographics_v003.csv,black_or_african_american,1.453642e+09,not_hispanic_or_latino,35.0,38.0
7,100000026,275.0,female,mspt_sociodemographics_v003.csv,white,1.462006e+09,not_hispanic_or_latino,32.0,32.0
...,...,...,...,...,...,...,...,...,...
201745,500003114,326.0,female,mspt_sociodemographics_v007.csv,western_europe,1.540558e+09,NaN,31.0,NaN
201746,500003116,326.0,female,mspt_sociodemographics_v007.csv,western_europe,1.541765e+09,NaN,50.0,60.0
201748,500003118,326.0,male,mspt_sociodemographics_v007.csv,western_europe,1.541756e+09,NaN,40.0,49.0
201749,500003119,396.0,female,mspt_sociodemographics_v007.csv,western_europe,1.527758e+09,NaN,42.0,42.0


In [8]:
for k, df in prepared_tables.items():
    print(k)
    display(df)

MSPT Sociodemographics

,mpi,site,sex,file,race,encounter_date,ethnicity,age_at_first_symptom,age_at_diagnosis
1,100000012,275.0,female,mspt_sociodemographics_v003.csv,white,1.461329e+09,not_hispanic_or_latino,40.0,40.0
4,100000017,275.0,female,mspt_sociodemographics_v003.csv,white,1.461402e+09,not_hispanic_or_latino,33.0,33.0
5,100000021,275.0,male,mspt_sociodemographics_v003.csv,white,1.461487e+09,not_hispanic_or_latino,30.0,30.0
6,100000023,275.0,male,mspt_sociodemographics_v003.csv,black_or_african_american,1.453642e+09,not_hispanic_or_latino,35.0,38.0
7,100000026,275.0,female,mspt_sociodemographics_v003.csv,white,1.462006e+09,not_hispanic_or_latino,32.0,32.0
...,...,...,...,...,...,...,...,...,...
201745,500003114,326.0,female,mspt_sociodemographics_v007.csv,western_europe,1.540558e+09,NaN,31.0,NaN
201746,500003116,326.0,female,mspt_sociodemographics_v007.csv,western_europe,1.541765e+09,NaN,50.0,60.0
201748,500003118,326.0,male,mspt_sociodemographics_v007.csv,western_europe,1.541756e+09,NaN,40.0,49.0
201749,500003119,396.0,female,mspt_sociodemographics_v007.csv,western_europe,1.527758e+09,NaN,42.0,42.0


MSPT Patientcharacteristics

,mpi,site,pdds_cat,pdds_scr,mstype,relapses,rel_stat,file,encounter_date,insurance,employment_status,current_living_situation,number_of_relapses,walking_aid,walking_aid_one_hand,years_of_education
2,100000010,275.0,moderate_disability,2.0,Relapsing Remitting MS,1,NaN,mspt_patient characteristics_v010.csv,1.461320e+09,medicaid,disabled,home_without_assistance,NaN,NaN,NaN,12.0
3,100000010,275.0,gait_disability,3.0,NaN,1,NaN,mspt_patient characteristics_v010.csv,1.471075e+09,medicare,disabled,home_with_assistance,NaN,NaN,NaN,NaN
4,100000010,275.0,mild_disability,1.0,NaN,0,NaN,mspt_patient characteristics_v010.csv,1.484397e+09,medicaid,disabled,home_without_assistance,NaN,NaN,NaN,NaN
5,100000010,275.0,mild_disability,1.0,NaN,0,NaN,mspt_patient characteristics_v010.csv,1.498912e+09,medicare,disabled,home_with_assistance,NaN,NaN,NaN,NaN
6,100000010,275.0,moderate_disability,2.0,NaN,0,NaN,mspt_patient characteristics_v010.csv,1.524225e+09,medicaid,disabled,home_without_assistance,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
725360,500013374,326.0,normal,0.0,Clinically Isolated Syndrome,0,domestic_partnership,mspt_patient_characteristics_v016.csv,1.631958e+09,free_social_healthcare,full_time,home_without_assistance,NaN,NaN,NaN,12.0
725410,500013406,326.0,normal,0.0,Clinically Isolated Syndrome,0,never_married,mspt_patient_characteristics_v016.csv,1.633686e+09,free_social_healthcare,full_time,home_without_assistance,NaN,NaN,NaN,16.0
725438,500013420,326.0,normal,0.0,Relapsing Remitting MS,0,married,mspt_patient_characteristics_v016.csv,1.633779e+09,free_social_healthcare,part_time,home_without_assistance,NaN,NaN,NaN,13.0
725785,500014006,326.0,mild_disability,1.0,Relapsing Remitting MS,0,married,mspt_patient_characteristics_v016.csv,1.642941e+09,free_social_healthcare,full_time,home_without_assistance,NaN,NaN,NaN,13.0


EMR Sociodemographics

...

,mpi,site,med_strt,med_end,med_code,med_name,med_dose,med_unit,file
7,100000010,275.0,1517547600.0,1.518858e+09,106346,MUPIROCIN 0.02 MG/MG TOPICAL OINTMENT,1.0,{APPLICATION},emr_medications_v006.csv
8,100000010,275.0,1479704400.0,1.482556e+09,1085636,TRIAMCINOLONE ACETONIDE 0.001 MG/MG TOPICAL OI...,NaN,NaN,emr_medications_v006.csv
9,100000010,275.0,1482555600.0,1.488172e+09,1085636,TRIAMCINOLONE ACETONIDE 0.001 MG/MG TOPICAL OI...,NaN,NaN,emr_medications_v006.csv
10,100000010,275.0,1488430800.0,1.498712e+09,1085636,TRIAMCINOLONE ACETONIDE 0.001 MG/MG TOPICAL OI...,NaN,NaN,emr_medications_v006.csv
11,100000010,275.0,1498712400.0,1.504328e+09,1085636,TRIAMCINOLONE ACETONIDE 0.001 MG/MG TOPICAL OI...,NaN,NaN,emr_medications_v006.csv
...,...,...,...,...,...,...,...,...,...
6869536,500001835,346.0,1438902000.0,NaN,1373484,TECFIDERA,480.0,MG,emr_medications_v005.csv
6869537,500001835,346.0,NaN,NaN,25480,GABAPENTIN,300.0,MG,emr_medications_v005.csv
6869538,500001835,346.0,NaN,NaN,E0143,"WALKER, FOLDING, WHEELED, ADJUSTABLE OR FIXED ...",NaN,NaN,emr_medications_v005.csv
6869546,500001849,346.0,1141686000.0,NaN,135779,COPAXONE,NaN,NaN,emr_medications_v005.csv


MRI Quantitative Metrics

,mpi,site,sty_date,pri_date,run_by,prog_ver,rad_acpt,bpf,bpf_chg,t2lesvol,...,scanner,dgmf,cgmf,thalf,t2overbv,t1dcpp,flrdcpp,nt2lesgt,file,study_id
0,100000010,275.0,1447977600.0,NaN,BIIB,MSPiev2.0.0rc.p1,NaN,0.795460,NaN,21.9116,...,Skyra,0.023679,0.373632,0.008384,0.023337,N,N,NaN,mri_quantitative_metrics_v015.csv,NaN
1,100000010,275.0,1461283200.0,1447977600.0,BIIB,MSPiev2.0.0rc.p1,NaN,0.798690,0.40605,20.4064,...,Skyra,0.023464,0.384924,0.008230,0.021645,N,N,1.0,mri_quantitative_metrics_v015.csv,NaN
2,100000010,275.0,1469318400.0,1461283200.0,BIIB,MSPiev2.0.0rc.p1,NaN,0.795880,-0.35183,20.6856,...,Skyra,0.023754,0.387206,0.008491,0.021805,N,N,0.0,mri_quantitative_metrics_v015.csv,NaN
3,100000010,275.0,1484352000.0,1469318400.0,BIIB,MSPiev2.0.0rc.p1,NaN,0.801200,0.66844,20.9312,...,Skyra,0.023400,0.376446,0.008233,0.022028,N,N,1.0,mri_quantitative_metrics_v015.csv,NaN
4,100000010,275.0,1496016000.0,1484352000.0,BIIB,MSPiev2.0.0rc.p1,NaN,0.805685,0.55979,20.0634,...,Skyra,0.023466,0.398746,0.008374,0.021117,N,N,0.0,mri_quantitative_metrics_v015.csv,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
200008,100030710,275.0,1531440000,1500163200.0,BIIB,MSPiev0.1.0a,NaN,0.888000,0.00600,2.1750,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,mri_quantitative_metrics_v007.csv,NaN
200009,100031255,275.0,1499990400,NaN,BIIB,MSPiev0.1.0a,NaN,0.826000,NaN,9.5660,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,mri_quantitative_metrics_v007.csv,NaN
200010,100031255,275.0,1527292800,1499990400.0,BIIB,MSPiev0.1.0a,NaN,0.826000,0.04200,10.6090,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,mri_quantitative_metrics_v007.csv,NaN
200011,100031762,275.0,1501372800,NaN,BIIB,MSPiev0.1.0a,NaN,0.870000,NaN,3.1430,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,mri_quantitative_metrics_v007.csv,NaN


Biobanking

,mpi,site,substudy,smk_code,smk_stat,specdate,specimen,subtype,lastmeal,visit_nm,disposit,visit_rn,file,site_id
1,100017491,235.0,2.0,266927001.0,Unknown if ever smoked,1.498157e+09,AP984771A01-005,DNA WB ALIQUOT,NaN,REDACTED,1.0,,biobanking_v015.csv,NaN
2,100061813,295.0,2.0,266927001.0,Unknown if ever smoked,1.570304e+09,AP850848D01,DNA WHOLE BLOOD,NaN,REDACTED,5.0,,biobanking_v015.csv,NaN
3,100039876,295.0,2.0,266927001.0,Unknown if ever smoked,1.547834e+09,AP909818C01-004,DNA WB ALIQUOT,NaN,REDACTED,1.0,,biobanking_v015.csv,NaN
5,100040242,295.0,2.0,266927001.0,Unknown if ever smoked,1.569602e+09,AP842989D01-003,DNA WB ALIQUOT,NaN,REDACTED,1.0,,biobanking_v015.csv,NaN
6,500001428,396.0,2.0,266927001.0,Unknown if ever smoked,1.568565e+09,EP176562E01,DNA WHOLE BLOOD,NaN,REDACTED,1.0,,biobanking_v015.csv,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2647151,100016824,235.0,2.0,449868002.0,Current every day smoker,1.497219e+09,AP992141A22,SERUM-MSPATHS-2,1.497208e+09,BASELINE,1.0,,biobanking_v012_CORRECTED.csv,NaN
2647152,100016824,235.0,2.0,449868002.0,Current every day smoker,1.497219e+09,AP992141A24,SERUM-BIOGEN-3,1.497208e+09,BASELINE,1.0,,biobanking_v012_CORRECTED.csv,NaN
2647153,100016824,235.0,2.0,449868002.0,Current every day smoker,1.497219e+09,AP992141A26,SERUM-BIOGEN-4,1.497208e+09,BASELINE,1.0,,biobanking_v012_CORRECTED.csv,NaN
2647154,100016824,235.0,2.0,449868002.0,Current every day smoker,1.497219e+09,AP992141A28,SERUM-MSPATHS-3,1.497208e+09,BASELINE,1.0,,biobanking_v012_CORRECTED.csv,NaN


Combine patients and HCs

In [9]:
mri_df = pd.concat((prepared_tables["MRI Quantitative Metrics"], prepared_tables_hc["MRI Quantitative Metrics"]))

# Convert Date-Column to same format everywhere
# 1. Identify which entries are numeric (Unix timestamps)
is_timestamp = pd.to_numeric(mri_df['sty_date'], errors='coerce').notna()

# 2. Convert timestamps (assumed to be seconds) to datetime
mri_df.loc[is_timestamp, 'date'] = pd.to_datetime(
    mri_df.loc[is_timestamp, 'sty_date'].astype(float), unit='s'
)

# 3. Convert the remaining string dates to datetime
mri_df.loc[~is_timestamp, 'date'] = pd.to_datetime(mri_df.loc[~is_timestamp, 'sty_date'])

# 4. Ensure the column dtype is datetime64[ns]
mri_df['date'] = pd.to_datetime(mri_df['date'])

mri_df = mri_df.merge(sessions_df, left_on=['mpi', 'date'], right_on=['subject', 'acq_time'], how='right').dropna(subset=['session_id', 'subject']).drop_duplicates(subset=['date', 'mpi'])
mri_df.query('session_id == "ses-001"').to_csv('baseline_quantiative_metrics.csv') 

mri_df

,mpi,site,sty_date,pri_date,run_by,prog_ver,rad_acpt,bpf,bpf_chg,t2lesvol,...,t2overbv,t1dcpp,flrdcpp,nt2lesgt,file,study_id,date,session_id,acq_time,subject
0,100052186,225.0,1556841600.0,NaN,BIIB,MSPiev2.0.0rc.p1,NaN,0.844728,NaN,38.2606,...,0.029835,N,N,NaN,mri_quantitative_metrics_v015.csv,NaN,2019-05-03,ses-001,2019-05-03,100052186
6,100052186,225.0,1582329600.0,1556841600.0,BIIB,MSPiev2.0.0,NaN,0.837414,-0.86593,33.4038,...,0.026759,N,N,8.0,mri_quantitative_metrics_v017.csv,NaN,2020-02-22,ses-002,2020-02-22,100052186
8,100006427,285.0,1486598400.0,NaN,BIIB,MSPiev2.0.0rc.p1,NaN,0.797080,NaN,34.3370,...,0.040321,N,N,NaN,mri_quantitative_metrics_v015.csv,NaN,2017-02-09,ses-001,2017-02-09,100006427
19,100006427,285.0,1496880000.0,1486598400.0,BIIB,MSPiev2.0.0rc.p1,NaN,0.801253,0.52353,33.2679,...,0.039202,N,N,0.0,mri_quantitative_metrics_v015.csv,NaN,2017-06-08,ses-002,2017-06-08,100006427
28,500002712,326.0,1604793600.0,NaN,BIIB,MSPiev2.0.0,NaN,0.788259,NaN,29.6135,...,0.028293,N,N,NaN,mri_quantitative_metrics_v015.csv,NaN,2020-11-08,ses-001,2020-11-08,500002712
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
193759,100003594,275.0,1511481600.0,1477612800.0,BIIB,MSPiev2.0.0rc.p1,NaN,0.856995,-0.18956,10.8606,...,0.009806,N,N,0.0,mri_quantitative_metrics_v015.csv,NaN,2017-11-24,ses-002,2017-11-24,100003594
193769,100003594,275.0,1577577600.0,1511481600.0,BIIB,MSPiev2.0.0rc.p1,NaN,0.854145,-0.33259,10.4541,...,0.009512,N,N,0.0,mri_quantitative_metrics_v015.csv,NaN,2019-12-29,ses-003,2019-12-29,100003594
193776,500003493,396.0,1560729600.0,NaN,BIIB,MSPiev2.0.0rc.p1,NaN,0.873024,NaN,0.1865,...,0.000172,Y,Y,NaN,mri_quantitative_metrics_v015.csv,NaN,2019-06-17,ses-001,2019-06-17,500003493
193782,500003493,396.0,1594339200.0,1560729600.0,BIIB,MSPiev2.0.0,NaN,0.869819,-0.36713,0.2813,...,0.000257,N,N,0.0,mri_quantitative_metrics_v015.csv,NaN,2020-07-10,ses-002,2020-07-10,500003493


Now match MRI-QUantiative values with our sessions. 

 => maybe we should reverse this. 
Currently we get 26782 sessions but have 30750 in the BIDS-Dataset => What happened here?
Should identify


## MSPT patient Characteristics

In This table we find essential things like MS-Type, PDDS-score, etc.



In [10]:
patchar_df = prepared_tables["MSPT Patientcharacteristics"]
patchar_df["date"] = pd.to_datetime(patchar_df.encounter_date, unit='s')
patchar_df.drop_duplicates(inplace=True)



def find_best_characteristics(mpi, date, progress, task):
    progress.update(task, advance=1)
    
    patient_df = patchar_df.query(f'mpi == "{mpi}"')

    patient_df["distance"] = abs(patient_df.date - date)
    patient_df.sort_values('distance', inplace=True)

    if len(patient_df) == 0:
        return  pd.Series([
        
            pd.NA, pd.NA, pd.NA
        ])

    
    best_result = patient_df.iloc[0,:]
    return pd.Series([
        best_result["date"],
        best_result["mstype"],
        best_result["pdds_scr"]
         ])



with Progress() as progress:
    task = progress.add_task("Matching Sessions", total=len(mri_df))
    mri_df[["characteristics_date", "mstype", "pdds_scr"]] = mri_df.apply(lambda x: find_best_characteristics(x.mpi, x.date, progress, task), axis=1 )
   

Output()

In [33]:
neurological_df = pd.concat((prepared_tables["MSPT Neurological"],prepared_tables_hc["MSPT Neurological"] ))

neurological_df["date"] = pd.to_datetime(neurological_df.encdate, unit='s')
neurological_df.drop_duplicates(inplace=True)



def find_best_characteristics(mpi, date, progress, task):
    progress.update(task, advance=1)
    
    patient_df = patchar_df.query(f'mpi == "{mpi}"')

    patient_df["distance"] = abs(patient_df.date - date)
    patient_df.sort_values('distance', inplace=True)

    if len(patient_df) == 0:
        return  pd.Series([
        
            pd.NA, pd.NA, pd.NA
        ])

    
    best_result = patient_df.iloc[0,:]
    return pd.Series([
        best_result["date"],
        best_result["mstype"],
        best_result["pdds_scr"]
         ])



# with Progress() as progress:
 #   task = progress.add_task("Matching Sessions", total=len(mri_df))
  #  mri_df[["characteristics_date", "mstype", "pdds_scr"]] = mri_df.apply(lambda x: find_best_characteristics(x.mpi, x.date, progress, task), axis=1 )
   


Now add Data from other Tables 

EMR Social History:
- smk_code
- smk_stat

    for HCs these are in Biobanking
- smk_code
 -smk_stat

MSPT patient characteristics
 - educ

mspt neurological assesments

## Add Smoking Status

In [30]:
biobanking_df = pd.concat((prepared_tables["Biobanking"], prepared_tables_hc["Biobanking"]))[["mpi", "smk_code", "smk_stat"]]
biobanking_df.drop_duplicates(inplace=True)


biobanking_df.groupby('mpi').agg('count')# .query('smk_code == 1')

,smk_code,smk_stat
mpi,,
100000010,3,3
100000011,3,3
100000012,2,2
100000016,2,2
100000017,2,2
...,...,...
500012902,2,2
500012925,3,3
500012943,2,2


In [ ]:
biobanking_df.query('mpi == "10000002"')

,mpi,smk_code,smk_stat
2418,100000024,8517006.0,Former smoker
4024,100000024,449868002.0,Current every day smoker
5756,100000024,266919005.0,Never smoker
17582,100000024,266927001.0,Unknown if ever smoked
384472,100000024,266927001.0,Tobacco smoking consumption unknown
387072,100000024,8517006.0,Ex-smoker
1861432,100000024,8517006.0,Former Smoker


In [ ]:
mri_df.to_hdf('mri_data.hdf5', key='mri_df')
